#Prepared by Tamal Acharya

In [ ]:
!pip install langchain-google-genai beautifulsoup4 pymupdf -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 727.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
import requests
from bs4 import BeautifulSoup
import io
from google.colab import userdata
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

def get_text_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text(separator='\n', strip=True)
        return text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

def get_text_from_pdf_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        pdf_content = response.content
        pdf_file = io.BytesIO(pdf_content)
        import fitz  # PyMuPDF
        doc = fitz.open(stream=pdf_file, filetype="pdf")
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching PDF URL: {e}")
        return None
    except ImportError:
        print("Please install pymupdf: !pip install pymupdf")
        return None
    except Exception as e:
        print(f"Error processing PDF: {e}")
        return None




In [ ]:
# Configure the Google Generative AI API
API_KEY = 'your_own_api_key'  #please use your own Gemini API key
genai.configure(api_key=API_KEY)

# Initialize the models
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001", google_api_key=API_KEY)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=API_KEY)



In [ ]:
# --- Simple RAG Implementation ---

def simple_rag_chat(query, context_text):
    """
    A very simple RAG function. It just concatenates the context and query
    and sends it to the LLM. No sophisticated retrieval or chunking.
    """
    if not context_text:
        print("No context available.")
        return llm.invoke(query).content # Just answer based on general knowledge

    prompt = f"Context: {context_text}\n\nBased on the context, answer the following query: {query}"
    response = llm.invoke(prompt).content
    return response



In [ ]:
# Example Usage:
# Replace with your desired URL (can be a web page or a direct link to a PDF)
# For a web page:
# url = "https://www.cnn.com/2023/11/20/india/india-cricket-world-cup-final-defeat-modi-intl-hnk/index.html"
# For a PDF:
url = "https://arxiv.org/pdf/2309.11519.pdf"

print(f"Fetching content from: {url}")

if url.lower().endswith('.pdf'):
    context_content = get_text_from_pdf_url(url)
else:
    context_content = get_text_from_url(url)


if context_content:
    print("Context fetched successfully. You can now ask questions about the content.")
    while True:
        user_query = input("Ask a question about the content (or type 'quit' to exit): ")
        if user_query.lower() == 'quit':
            break
        if user_query.strip():
            answer = simple_rag_chat(user_query, context_content)
            print("\nAnswer:")
            print(answer)
            print("-" * 20)
else:
    print("Failed to fetch content. Cannot answer questions based on this URL.")

Fetching content from: https://arxiv.org/pdf/2309.11519.pdf
Context fetched successfully. You can now ask questions about the content.
Ask a question about the content (or type 'quit' to exit): summarise the paper

Answer:
This paper presents an optimization strategy for waveguide-based multi-quantum well (MQW) balanced photodetectors (BPDs) operating at 1550 nm on a generic InP platform for high-speed coherent receivers. The design parameters of the BPD are optimized for maximum bandwidth at a specific responsivity using a Particle Swarm Optimization (PSO) algorithm. The optimization considers the trade-off between carrier transit time and RC loading. The paper also discusses the integration of the optimized BPD with a laser for coherent receiver implementation, addressing the challenges of balancing BPD performance with laser requirements on the same platform. The methodology aims to provide multiple design options that can be tailored to specific fabrication constraints and performa